In [16]:
import pandas as pd
import geopandas as gpd
import openrouteservice as ors
import folium
import sys
import concurrent.futures
from tqdm import tqdm
from threading import Semaphore
import json
import folium
import openrouteservice
import json
import time
import openrouteservice
import geopandas as gpd
import folium
from shapely.geometry import Point
from shapely.geometry import shape
from geopy.distance import geodesic
from shapely import wkt
from openrouteservice.exceptions import ApiError
sys.path.append('../scripts')
from utilities import get_vic_subs_as_csv

In [17]:
# client = ors.Client(key='5b3ce3597851110001cf624828f2b63314b24af798ed8608d8464d19')
client = ors.Client(key='5b3ce3597851110001cf62485538e6b62c8b41f7a68feb510f25a05e')


In [11]:
input_file = '../data/curated/m_curr_pred_median.csv'
output_file = '../data/curated/house_transport_part_2.csv'
with open(input_file, 'r') as file:
    data = pd.read_csv(file)
    

In [12]:
# Define the rate limit
RATE_LIMIT = 100  # Requests per minute
REQUEST_INTERVAL = 60 / RATE_LIMIT  # Interval between requests in seconds

# Semaphore to control the number of concurrent requests
semaphore = Semaphore(3)  # Max number of concurrent threads
def get_coordinates(address):
    try:
        with semaphore:
            response = client.pelias_search(address)
            if response['features']:
                coords = response['features'][0]['geometry']['coordinates']
                return {'latitude': coords[1], 'longitude': coords[0]}
            else:
                return {'latitude': None, 'longitude': None}
    except Exception as e:
        print(f"Error fetching coordinates for address '{address}': {e}")
        return (coords[1], coords[0])

def worker(row):
    address = row.get('address') 
    if address:
        coords = get_coordinates(address)
        return coords
    return None

In [13]:

df = pd.read_csv(input_file)

# Add a column 'geometry' to store (latitude, longitude) tuples
df['geometry'] = None


In [ ]:
# Find coordinates based on address for houses
max_threads = 3

with concurrent.futures.ThreadPoolExecutor(max_threads) as executor:
    # Create a list of futures for each row
    futures = {executor.submit(worker, row): index for index, row in list(df.iterrows())[3500:]}

    # Wrap futures with tqdm to show progress
    for future in tqdm(concurrent.futures.as_completed(futures), total=len(futures), desc="Processing items"):
        index = futures[future]
        try:
            # Update the 'geometry' column for each row
            df.at[index, 'geometry'] = future.result()
        except Exception as e:
            print(f"An error occurred at index {index}: {e}")


In [ ]:
df.to_csv(output_file, index=False)

print(f"Updated data with coordinates saved to {output_file}")

In [19]:
housing_file_1 = '../data/curated/house_transport_part_2.csv'
housing_file_2 = '../data/curated/house_transport.csv'
df1 = pd.read_csv(housing_file_1)
df2 = pd.read_csv(housing_file_2)

# Ensure the geometry column is updated by filling missing values from the second DataFrame
# This assumes the 'address' column is the same in both files and is used for merging
df1['geometry'] = df1['geometry'].combine_first(df2['geometry'])

In [20]:
merged_file = '../data/curated/merged_houses_with_complete_geometry.csv'
df1.to_csv(merged_file, index=False)

print(f"Merged file saved to: {merged_file}")

Merged file saved to: ../data/curated/merged_houses_with_complete_geometry.csv


In [23]:
houses_coordinates = []

for index, row in df1.iterrows():
    geometry = row['geometry']
    
    # Check if geometry is in dictionary format with 'latitude' and 'longitude' keys
    if isinstance(geometry, str):
        try:
            # Convert string representation of dictionary to actual dictionary
            geometry_dict = eval(geometry)
            
            if 'latitude' in geometry_dict and 'longitude' in geometry_dict:
                latitude = geometry_dict['latitude']
                longitude = geometry_dict['longitude']
                houses_coordinates.append({
                    'address': row['address'],
                    'coordinates': (latitude, longitude)
                })
            else:
                print(f"Missing latitude or longitude for house: {row['address']}, geometry: {geometry}")
        
        except Exception as e:
            print(f"Error parsing geometry for house: {row['address']}, error: {e}")
    else:
        print(f"Invalid geometry format for house: {row['address']}, geometry: {geometry}")

# Now houses_coordinates contains all the valid house addresses and their coordinates


In [ ]:
houses_coordinates

In [25]:
gdf_regional = gpd.read_file("../data/PTV/", layer='PTV_REGIONAL_TRAIN_STATION')
gdf_regional = gdf_regional.to_crs(epsg=4326)

gdf_metro = gpd.read_file("../data/PTV/", layer='PTV_METRO_TRAIN_STATION')
gdf_metro = gdf_metro.to_crs(epsg=4326)


In [27]:
gdf_tram = gpd.read_file("../data/PTV_tram_bus/", layer='PTV_METRO_TRAM_STOP')
gdf_tram = gdf_tram.to_crs(epsg=4326)




In [28]:
def prepare_station_data(gdf):
    return gdf[['geometry', 'STOP_ID', 'STOP_NAME']].apply(
        lambda row: ((row['geometry'].y, row['geometry'].x), row['STOP_ID'], row['STOP_NAME']), axis=1
    ).tolist()

In [29]:
train_station_gdf = gpd.GeoDataFrame(pd.concat([gdf_metro, gdf_regional], ignore_index=True))
tram_station_gdf = gpd.GeoDataFrame(gdf_tram)

train_station_data = prepare_station_data(train_station_gdf)
tram_station_data = prepare_station_data(tram_station_gdf)


In [32]:
housing_file = '../data/curated/merged_houses_with_complete_geometry.csv'
houses_df = pd.read_csv(housing_file)

# Convert 'geometry' column to a usable format
houses_df['geometry'] = houses_df['geometry'].apply(eval) 


In [ ]:
houses_df

In [34]:
def find_closest_station(house_coords, stations):
    closest_station = None
    min_distance = float('inf')
    closest_station_id = None
    closest_station_name = None
    
    for (station_coords, station_id, station_name) in stations:
        distance = geodesic(house_coords, station_coords).kilometers
        if distance < min_distance:
            min_distance = distance
            closest_station = station_coords
            closest_station_id = station_id
            closest_station_name = station_name
            
    return closest_station, closest_station_id, closest_station_name, min_distance

In [35]:
for idx, row in houses_df.iterrows():
    house_coords = (row['geometry']['latitude'], row['geometry']['longitude'])
    
    # Find the closest train station
    closest_train_station_coords, closest_train_station_id, closest_train_station_name, train_distance = find_closest_station(house_coords, train_station_data)
    
    # Find the closest tram station
    closest_tram_station_coords, closest_tram_station_id, closest_tram_station_name, tram_distance = find_closest_station(house_coords, tram_station_data)
    
    # Update the DataFrame with closest station data
    houses_df.at[idx, 'closest_train_station_name'] = closest_train_station_name
    houses_df.at[idx, 'closest_train_station_distance_km'] = train_distance
    houses_df.at[idx, 'closest_tram_station_name'] = closest_tram_station_name
    houses_df.at[idx, 'closest_tram_station_distance_km'] = tram_distance
    
    # Print progress every 1000 houses
    if (idx + 1) % 1000 == 0:
        print(f"Processed {idx + 1} houses...")

# Save the updated DataFrame to a new CSV file
output_file = '../data/curated/houses_with_closest_stations.csv'
houses_df.to_csv(output_file, index=False)

print(f"Merged data with closest stations saved to: {output_file}")

Processed 1000 houses...
Processed 2000 houses...
Processed 3000 houses...
Processed 4000 houses...
Processed 5000 houses...
Processed 6000 houses...
Merged data with closest stations saved to: ../data/curated/houses_with_closest_stations.csv


In [37]:
def find_closest_location(house_coords, locations):
    closest_location = None
    min_distance = float('inf')
    closest_location_id = None
    closest_location_name = None
    
    for (location_coords, location_id, location_name) in locations:
        distance = geodesic(house_coords, location_coords).kilometers
        if distance < min_distance:
            min_distance = distance
            closest_location = location_coords
            closest_location_id = location_id
            closest_location_name = location_name
            
    return closest_location, closest_location_id, closest_location_name, min_distance

In [39]:
hospital_file = '../data/raw/vic_hospitals_with_coordinates.csv'
hospitals_df = pd.read_csv(hospital_file)

# Convert 'geometry' column from string to list format
hospitals_df['geometry'] = hospitals_df['geometry'].apply(eval)

# Create a list of hospital coordinates, IDs, and names for proximity search
hospital_data = [((row['geometry'][1], row['geometry'][0]), row['Establishment ID'], row['Hospital name']) 
                 for idx, row in hospitals_df.iterrows()]

In [ ]:
for idx, row in houses_df.iterrows():
    house_coords = (row['geometry']['latitude'], row['geometry']['longitude'])
    
    # Find closest hospital
    closest_hospital_coords, closest_hospital_id, closest_hospital_name, hospital_distance = find_closest_location(house_coords, hospital_data)
    
    # Update DataFrame with closest hospital data
    houses_df.at[idx, 'closest_hospital_name'] = closest_hospital_name
    houses_df.at[idx, 'closest_hospital_distance_km'] = hospital_distance
    
    # Print progress every 1000 houses
    if (idx + 1) % 1000 == 0:
        print(f"Processed {idx + 1} houses...")

# Save the updated DataFrame to a new CSV file
output_file = '../data/curated/houses_with_closest_stations_and_hospital.csv'
houses_df.to_csv(output_file, index=False)

print(f"Data with closest hospitals saved to: {output_file}")

In [41]:
houses_df = houses_df.drop(columns=['2025', '2026', '2027'])

# Save the updated DataFrame to a new CSV file
output_file = '../data/curated/houses_with_closest_stations_and_hospital.csv'
houses_df.to_csv(output_file, index=False)

print(f"Data with dropped columns saved to: {output_file}")

Data with dropped columns saved to: ../data/curated/houses_with_closest_stations_and_hospital.csv
